## 导入所需包

In [1]:
import torch,os
from torch.utils.data import DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter

## 分别下载训练和测试数据集，并生成训练加载

In [2]:
root = os.getcwd()
batch_size = 100
# 加载数据时即将其转换为tensor并归一化
train_dataset = dsets.MNIST(root, train=True, download=True,
                            transform=transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))
test_dataset = dsets.MNIST(root, train=False, download=True, 
                            transform=transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

# 生成训练loader和测试loader
train_loader = DataLoader(dataset=train_dataset,
                           batch_size=batch_size,
                             shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                           batch_size=batch_size,
                            shuffle=True)

## LeNet-5网络的定义

In [3]:
# LeNet-5
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        # 由于MNIST的大小为（1，28，28），所以需要微调第一层卷积，以适应大小，并并影响后续的大小
        self.layer1 = nn.Sequential(nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
                                    nn.BatchNorm2d(6),
                                    nn.Sigmoid(),
                                    nn.AvgPool2d(kernel_size=2, stride=2))
                                    #nn.ReLU(),
                                    #nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                                    nn.BatchNorm2d(16),
                                    nn.Sigmoid(),
                                    nn.AvgPool2d(kernel_size=2, stride=2))
                                    #nn.ReLU(),
                                    #nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc1 = nn.Sequential(nn.Linear(5 * 5 * 16, 120),
                                 nn.Sigmoid(),
                                 #nn.ReLU()
                                )
        self.fc2 = nn.Sequential(nn.Linear(120, 84),
                                       nn.Sigmoid(),
                                       #nn.ReLU()
                                        )
        self.fc3 = nn.Linear(84, num_classes)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

## 定义超参数和优化器

In [4]:
# 设置训练的epoch和类别、学习率
num_epoches = 15
num_classes = 10
learning_rate = 0.001

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = LeNet5(num_classes).to(device)

# 打印网络结构
print(summary(net, input_size=(1, 1, 28, 28)))

# 使用交叉熵损失函数和Adam优化器
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
# optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

Layer (type:depth-idx)                   Output Shape              Param #
LeNet5                                   [1, 10]                   --
├─Sequential: 1-1                        [1, 6, 14, 14]            --
│    └─Conv2d: 2-1                       [1, 6, 28, 28]            156
│    └─BatchNorm2d: 2-2                  [1, 6, 28, 28]            12
│    └─Sigmoid: 2-3                      [1, 6, 28, 28]            --
│    └─AvgPool2d: 2-4                    [1, 6, 14, 14]            --
├─Sequential: 1-2                        [1, 16, 5, 5]             --
│    └─Conv2d: 2-5                       [1, 16, 10, 10]           2,416
│    └─BatchNorm2d: 2-6                  [1, 16, 10, 10]           32
│    └─Sigmoid: 2-7                      [1, 16, 10, 10]           --
│    └─AvgPool2d: 2-8                    [1, 16, 5, 5]             --
├─Sequential: 1-3                        [1, 120]                  --
│    └─Linear: 2-9                       [1, 120]                  48,120
│    └─

## 开始训练

In [ ]:
writer = SummaryWriter('./logs')
for epoch in range(num_epoches):
    print('current epoch = %d' % epoch)
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images) 
        
        # 训练损失
        train_loss = criterion(outputs, labels) 
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print('current loss = %.5f' % train_loss.item())
        
        # 训练准确率
        _, predicts = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicts == labels).sum()
        train_accuracy = correct / total
        if epoch == num_epoches-1 and i == len(train_loader)-1:
            print('train_accuracy = %.2f' % (100 * train_accuracy) + '%')

    # 测试准确率
    eval_total=0
    eval_correct=0
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images) 
        _, pred = torch.max(outputs.data, 1)
        eval_total += labels.size(0)
        eval_correct += (pred == labels).sum()
        test_accuracy = eval_correct / eval_total

    writer.add_scalar("loss", train_loss, epoch)
    writer.add_scalar("train accuracy", train_accuracy, epoch)
    writer.add_scalar("test accuracy", test_accuracy, epoch)
print('Accuracy = %.2f' % (100 * eval_correct / eval_total) + '%')

writer.close()

current epoch = 0
current loss = 2.35197
current loss = 1.74586
current loss = 0.98308
current loss = 0.63945
current loss = 0.41895
current loss = 0.38313
current epoch = 1
current loss = 0.20530
current loss = 0.19915
current loss = 0.18299
current loss = 0.15109
current loss = 0.12345
current loss = 0.11321
current epoch = 2
current loss = 0.09068
current loss = 0.07689
current loss = 0.07936
current loss = 0.10912
current loss = 0.06827
current loss = 0.03424
current epoch = 3
current loss = 0.07554
current loss = 0.05858
current loss = 0.06346
current loss = 0.15858
current loss = 0.10490
current loss = 0.11054
current epoch = 4
current loss = 0.11184
current loss = 0.05218
current loss = 0.12201
current loss = 0.07818
current loss = 0.07085
current loss = 0.08982
current epoch = 5
current loss = 0.04784
current loss = 0.05594
current loss = 0.08808
current loss = 0.10677
current loss = 0.02075
current loss = 0.07253
current epoch = 6
current loss = 0.01632
current loss = 0.02970
